#  Basic Model Blender

# 0. Preparation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

In [2]:
train = pd.read_csv("train.csv")
train.head()

C:\Users\yshha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,企业类型,经营期限至,登记机关,企业状态,邮政编码,投资总额,注册资本,核准日期,行业代码,...,应收账款_年末数,一年内到期的非流动资产_年末数,油气资产_年末数,应收补贴款_年末数,应收股利_年末数,应收利息_年末数,应收票据_年末数,预付款项_年末数,资产总计_年末数,在建工程_年末数
0,1,9.0,NaN,1.0,0.0,266300,19.110000,19.110000,00:00.0,5154.0,...,142.310005,0.0,0.0,0.0,0.0,0.0,28.122902,1.004419,632.032182,0.0
1,3,11.0,NaN,5.0,0.0,266000,29.154194,19.210446,00:00.0,7214.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,0.0,NaN,3.0,0.0,266000,NaN,19.120045,13:31.0,2110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,0.0,42:05.0,9.0,0.0,266000,NaN,19.160223,00:00.0,5163.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14,0.0,NaN,2.0,0.0,266400,NaN,19.210446,00:00.0,5439.0,...,156.901030,0.0,0.0,0.0,0.0,0.0,10.044194,0.508236,319.205293,0.0


In [3]:
test = pd.read_csv("test.csv")

In [4]:
y_train = pd.read_csv("train_label.csv")
y_train = y_train.Label

In [5]:
# get the numeric column names
col_num = train.select_dtypes(include = "number").columns.tolist()
col_num.remove("ID")
print(len(col_num))

# get the objective column names
col_obj = train.select_dtypes(include = "object").columns.tolist()
print(len(col_obj))

170
7


In [6]:
categorical = ['企业类型', '登记机关', '企业状态', '行业代码', '行业门类', '企业类别', '管辖机关']
numerical = list(set(col_num).difference(set(categorical)))
categorical = ['企业类别', '企业状态']

# 1 Feature Engineering

## 1.1 Numerical

In [7]:
x_begin = [col for col in numerical if "年初数" in col]
print(len(x_begin))

x_end = [col for col in numerical if "年末数" in col]
print(len(x_end))

x =  [col[:-4] for col in x_begin]

78
78


In [8]:
for v in x:
    train[v + "_年增量"] = train[v + "_年末数"] - train[v + "_年初数"]
    test[v + "_年增量"] = test[v + "_年末数"] - test[v + "_年初数"]                                        

In [9]:
numerical = list(set(numerical).difference(set(x_begin)))
numerical = numerical + [v + "_年增量" for v in x]

## 1.2 Categorical(“企业类别”): one-hot encoding

In [10]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

dummies_train = pd.get_dummies(train["企业类别"],
                               prefix = "企业类别", drop_first=False)
dummies_test = pd.get_dummies(test["企业类别"],
                              prefix = "企业类别", drop_first=False)

dummies_train1 = pd.get_dummies(train["管辖机关"],
                               prefix = "管辖机关", drop_first=False)
dummies_test1 = pd.get_dummies(test["管辖机关"],
                              prefix = "管辖机关", drop_first=False)

dummies_train2 = pd.get_dummies(train["行业门类"],
                               prefix = "行业门类", drop_first=False)
dummies_train2['行业门类_19.0'] = 0
dummies_test2 = pd.get_dummies(test["行业门类"],
                              prefix = "行业门类", drop_first=False)

X_train = train[numerical + ["企业状态"]].join(dummies_train).join(dummies_train1).join(dummies_train2)
X_test = test[numerical + ["企业状态"]].join(dummies_test).join(dummies_train1).join(dummies_train2)


## 1.3 Filling NA with Multiple Imputation Branch

In [11]:
# convert none to NA
X_train = X_train.as_matrix().astype(np.float)
X_train = pd.DataFrame(X_train)
X_test = X_test.as_matrix().astype(np.float)
X_test = pd.DataFrame(X_test)

C:\Users\yshha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\yshha\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [12]:
# fill NA with MICE
imp = IterativeImputer()
X_train = pd.DataFrame(imp.fit_transform(X_train))
X_test = pd.DataFrame(imp.fit_transform(X_test))

C:\Users\yshha\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:599: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\yshha\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:599: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [13]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,0.000000,532.218401,0.000000,489.934244,0.000000,0.000000,0.000000,585.013852,0.000000,3.181189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,337.586649,4041.172913,11.870128,693.913429,83.812029,-1.110551,9.892151,3204.118314,0.155419,1.175269,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,296.135822,4041.549397,12.026712,694.593448,83.115452,2.722681,9.534785,3202.989380,42.640651,4.637491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,285.374426,4041.065889,11.801505,696.076124,84.188191,1.051914,8.837440,3204.070762,41.765796,0.001318,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,316.101911,0.000000,51.730490,0.000000,0.000000,0.000000,322.318888,0.000000,0.893524,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1.4 Scale Based on Sparse Matrix

In [14]:
from sklearn.preprocessing import PowerTransformer

Scaler = PowerTransformer()
Scaler.fit_transform(X_train)
X_train = Scaler.transform(X_train)
X_test = Scaler.transform(X_test)

C:\Users\yshha\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:2863: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


## 1.6 Basic Models with Default Parameters

In [ ]:
%%time

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# svc
scores = ['precision', 'recall']
tuned_parameters = {
    'kernel':['rbf','sigmoid','linear'],'gamma':[1e-2,1e-3,1e-4],'C':[1,10,100,1000]}
svc = GridSearchCV(SVC(), tuned_parameters, cv = 5, scoring ='%s_macro' % scores[0])
svc.fit(X_train, y_train)
print(svc.best_score_)

# mlp
mlp = MLPClassifier(max_iter=100)
tune_parameters = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
    }
mlp_up = GridSearchCV(mlp, tune_parameters , cv = 5)
mlp_up.fit(X_train, y_train)
print(mlp_up.best_score_)

C:\Users\yshha\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\yshha\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\yshha\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\yshha\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\yshha\Anaconda3\lib\site-packages\sklearn\m

In [148]:
# blend models based on scores above
def blend_models(x):
    svc_pred =  svc.predict_proba(x)
    mlp_pred =  mlp_up.predict_proba(x)
    return ((0.5 * svc_pred) + (0.5 * mlp_pred))

In [149]:
submission = pd.read_csv("submission_example.csv")
y_pred = blend_models(X_test)
submission["Label"] = 1-y_pred
submission.to_csv("submission_1.csv", index = False)

C:\Users\yshha\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
